## 참고자료
`https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html`

## Import Required Libraries

In [1]:
import random
import torch
import torch.nn as nn
from torch import optim

## GPU 설정 및 Random Seed 설정

In [2]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Raw Data 생성 (Korean >> English)

In [3]:
raw = ["I feel hungry.	나는 배가 고프다.",
       "Pytorch is very easy.	파이토치는 매우 쉽다.",
       "Pytorch is a framework for deep learning.	파이토치는 딥러닝을 위한 프레임워크이다.",
       "Pytorch is very clear to use.	파이토치는 사용하기 매우 직관적이다."]

## 문장의 시작과 끝을 알려줄 Token 만들기

In [4]:
# fix token for "start of sentence" and "end of sentence"
SOS_token = 0
EOS_token = 1

## Vocab Class 만들기

- 문장의 시작, 의미 있는 단어들이 담겨있는 딕션너리 만들기

In [5]:
# 의미있는 data를 담고 있는 Vocabulary
class Vocab:
    def __init__(self):
        self.vocab2index = {"<SOS>":SOS_token, "<EOS>": EOS_token}
        self.index2vocab = {SOS_token:"<SOS>" , EOS_token :"<EOS>"}
        self.vocab_count = {}                 # 단어의 갯수
        self.n_vocab = len(self.vocab2index)  # 단어의 길이
        
    def add_vocab(self, sentense):
        for word in sentense.split(" "):
            # 만약 문장의 시작이나 끝인지 아닌지 검사
            if word not in self.vocab2index:          # 만약 새로운 단어가 들어 왔다면,
                self.vocab2index[word] = self.n_vocab  # 단어의 길이를 index로 사용
                self.vocab_count[word] = 1             # 단어가 몇 개 있는지 count
                self.index2vocab[self.n_vocab] = word  # 해당 index에 단어를 넣는다.
                self.n_vocab += 1                      # 단어의 길이 + 1
            
            else:                                      # 만약 vocab2index에 있는 단어라면
                self.vocab_count[word] += 1            # 해당 단어 count + 1

## Source data(English)와 Target data(Korean) 나누는 함수 구현

In [6]:
# filter out the long sentence from source and target data
def filter_pair(pair, source_max_length, target_max_length):
    return len(pair[0].split(" ")) < source_max_length and len(pair[1].split(" ")) < target_max_length
# source word의 길이 또는 max 값 중 하나 반환 그리고 target word의 길이 또는 max 값 중 하나 반환 (총 2개 반환)

## Raw Data를 Source data와 Target data로 나누는 함수 구현

In [7]:
# read and preprocess the corpus data
# raw data, source_max_length, target_max_length
def preprocess(corpus, source_max_length, target_max_length):
    print("reading corpus...")
    pairs = []
    for line in corpus:
        pairs.append([s for s in line.strip().lower().split("\t")])  
        # strip() - 양 끝에 있는 공백을 날리는 함수
        # lower() - 대문자를 소문자로 바꾸는 함수
        # split() - 문자열을 나눌때 사용하는 함수
    print("Read {} sentence pairs".format(len(pairs)))

    pairs = [pair for pair in pairs if filter_pair(pair, source_max_length, target_max_length)]
    print("Trimmed to {} sentence pairs".format(len(pairs)))
    
    # vocab 클래스 생성
    source_vocab = Vocab()
    target_vocab = Vocab()

    print("Counting words...")
    for pair in pairs:
        source_vocab.add_vocab(pair[0])
        target_vocab.add_vocab(pair[1])
    print("source vocab size =", source_vocab.n_vocab)
    print("target vocab size =", target_vocab.n_vocab)

    return pairs, source_vocab, target_vocab

## Encoder 정의 (Input data >> 의미 있는 벡터)

RNN 모델 중 GRU 사용

In [8]:
# declare simple encoder
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)  # 거대한 행렬
        # 임베딩 - 해당 index의 단어를 벡터의 조합으로 표현 (input_size - 단어의 갯수, hidden_size - 임베딩할 벡터의 차원)
        self.gru = nn.GRU(hidden_size, hidden_size)   # RNN 모델 중 GRU 모델 사용

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)  # GRU로 넣기 위한 Tensor의 모형 변환
        x, hidden = self.gru(x, hidden)
        return x, hidden

## Decoder 정의 (의미 있는 벡터 >> Output data)

In [9]:
# declare simple decoder
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size  # 의미있는 벡터 (Input)
        self.embedding = nn.Embedding(output_size, hidden_size)
        # 임베딩 - 해당 index의 단어를 벡터의 조합으로 표현 (input_size - 단어의 갯수, hidden_size - 임베딩할 벡터의 차원)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)   # FC layer
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        x = self.embedding(x).view(1, 1, -1)
        x, hidden = self.gru(x, hidden)    # x.shpae = (1,1,16)
        x = self.softmax(self.out(x[0]))
        return x, hidden

## 문장을 Index로 바꾼 후 Tensor로 바꾸는 함수 구현

In [10]:
# convert sentence to the index tensor
def tensorize(vocab, sentence):
    indexes = [vocab.vocab2index[word] for word in sentence.split(" ")]  # 단어를 index로 변환
    indexes.append(vocab.vocab2index["<EOS>"])                            # EOS를 추가하여 끝을 알림.
    return torch.Tensor(indexes).long().to(device).view(-1, 1)

## Training 함수 구현
 

In [11]:
# training seq2seq
def train(pairs, source_vocab, target_vocab, encoder, decoder, n_iter, print_every=1000, learning_rate=0.01):
        
    loss_total = 0
    
    # optimzer 설정
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_batch = [random.choice(pairs) for _ in range(n_iter)]
    training_source = [tensorize(source_vocab, pair[0]) for pair in training_batch]
    training_target = [tensorize(target_vocab, pair[1]) for pair in training_batch]

    criterion = nn.NLLLoss()   # Negatice Log Liklihood

    for i in range(1, n_iter + 1):
        source_tensor = training_source[i - 1]  # input data 만들기
        target_tensor = training_target[i - 1]  # ouput data 만들기

        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)  # encoder ouput vector 만들기
    
        # source의 길이 , target의 길이
        source_length = source_tensor.size(0)
        target_length = target_tensor.size(0)

        loss = 0
        
        # source에서 단어 하나를 가져와서 Encoder를 만든다.
        for enc_input in range(source_length):
            _, encoder_hidden = encoder(source_tensor[enc_input], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)
        decoder_hidden = encoder_hidden # connect encoder output to decoder input
        
        # Encoder에서 나온 벡터와 target을 이용하여 Loss 계산
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # teacher forcing
            # 그 다음 layer부터는 전 RNN의 결과값을 사용해서 input을 만들어 낸다.
        
        # 기울기 초기화
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        # backpropagation
        loss.backward()
        
        # weight update
        encoder_optimizer.step()
        decoder_optimizer.step()

        loss_iter = loss.item() / target_length
        loss_total += loss_iter
        
        # print_every마다 loss값 출력
        if i % print_every == 0:
            loss_avg = loss_total / print_every
            loss_total = 0
            print("[{} - {}%] loss = {:05.4f}".format(i, i / n_iter * 100, loss_avg))

## 평가(Test)하는 함수 구현

In [12]:
# insert given sentence to check the training
def evaluate(pairs, source_vocab, target_vocab, encoder, decoder, target_max_length):
    for pair in pairs:
        print(">", pair[0])  # source vocab 출력
        print("=", pair[1])  # target vocab 출력
        source_tensor = tensorize(source_vocab, pair[0])  # tensor로 바꾸기
        source_length = source_tensor.size()[0]           # source_length 길이 구하기
        encoder_hidden = torch.zeros([1, 1, encoder.hidden_size]).to(device)    # to(device)로 Tensor를 무엇으로 돌리는지 알려주기
        
        # 각 source vocab마다 Encoder를 돌린다.
        for ei in range(source_length):
            _, encoder_hidden = encoder(source_tensor[ei], encoder_hidden)

        decoder_input = torch.Tensor([[SOS_token]]).long().to(device)           # to(device)로 Tensor를 무엇으로 돌리는지 알려주기
        decoder_hidden = encoder_hidden
        decoded_words = []
        
        # 각 target vocab마다 Decoder를 돌린다.
        for di in range(target_max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            _, top_index = decoder_output.data.topk(1)    # ouput data에서 top 1 의 data만 받는다.
            if top_index.item() == EOS_token:             # 문장이 끝날 때까지 반복
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(target_vocab.index2vocab[top_index.item()])

            decoder_input = top_index.squeeze().detach()  # gradient 전파가 안되는 tensor 생성

        predict_words = decoded_words
        predict_sentence = " ".join(predict_words)
        print("<", predict_sentence)
        print("")

## Hyper parameter 정의

In [13]:
# declare max length for sentence
SOURCE_MAX_LENGTH = 10
TARGET_MAX_LENGTH = 12

In [14]:
# preprocess the corpus
load_pairs, load_source_vocab, load_target_vocab = preprocess(raw, SOURCE_MAX_LENGTH, TARGET_MAX_LENGTH)
print(random.choice(load_pairs))

reading corpus...
Read 4 sentence pairs
Trimmed to 4 sentence pairs
Counting words...
source vocab size = 17
target vocab size = 13
['pytorch is very easy.', '파이토치는 매우 쉽다.']


## Encoder, Decoder 정의

In [15]:
# declare the encoder and the decoder
enc_hidden_size = 16
dec_hidden_size = enc_hidden_size
enc = Encoder(load_source_vocab.n_vocab, enc_hidden_size).to(device)
dec = Decoder(dec_hidden_size, load_target_vocab.n_vocab).to(device)

## Training

In [16]:
# train seq2seq model
train(load_pairs, load_source_vocab, load_target_vocab, enc, dec, 5000, print_every=1000)

[1000 - 20.0%] loss = 0.7431
[2000 - 40.0%] loss = 0.1105
[3000 - 60.0%] loss = 0.0341
[4000 - 80.0%] loss = 0.0185
[5000 - 100.0%] loss = 0.0125


## Test

In [17]:
# check the model with given data
evaluate(load_pairs, load_source_vocab, load_target_vocab, enc, dec, TARGET_MAX_LENGTH)

> i feel hungry.
= 나는 배가 고프다.
< 나는 배가 고프다. <EOS>

> pytorch is very easy.
= 파이토치는 매우 쉽다.
< 파이토치는 매우 쉽다. <EOS>

> pytorch is a framework for deep learning.
= 파이토치는 딥러닝을 위한 프레임워크이다.
< 파이토치는 딥러닝을 위한 프레임워크이다. <EOS>

> pytorch is very clear to use.
= 파이토치는 사용하기 매우 직관적이다.
< 파이토치는 사용하기 매우 직관적이다. <EOS>

